In [31]:
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import requests
from xml.etree import ElementTree as ET
from io import StringIO

# 05-Compare holdings
The purpose of this document is to accept a list of securities by Series ID then return a list of how much they overlap.

Psudeo code:

- Import each filing
- Join on CUSIP (change column name to match security set.
- Add rows
- graph

<hr>
Import functions from previous

In [2]:
# From 02-NPORT function.ipynb
class download_NPORT:
    def __init__(self, link):
        r = requests.get(link)
        assert(r), r
        xml = ET.fromstring(r.text)
        
        # Remove namespace tags
        xml = ET.iterparse(StringIO(r.text))
        for _, el in xml:
            _, _, el.tag = el.tag.rpartition('}') # strip ns
        self.xml = xml.root
    def parseSecurities(self):
        secs = self.xml.find('formData').find('invstOrSecs')
        all_secs = []
        for s in secs:
            t_dict = {}
            for e in s:
                t_dict[e.tag] = e.text 
            all_secs += [t_dict]
        res = pd.DataFrame(all_secs)
                
        # Extra data cleaning to convert strings to numbers
        num_cols = ['valUSD','balance','pctVal']
        
        for c in num_cols:
            res[c] = res[c].astype(float)
        
        return res
    def parseFundInfo(self):
        fund_info = {}
        for e in list(self.xml.find('formData').find('fundInfo')):
            fund_info[e.tag] = e.text
        
        return fund_info
    
# From 03-EDGAR getall
## NOTE: Although not expressly later int he document, the functions call other functions.
## All are necessary.
def get_EDGAR_index(year,quarter):
    assert(int(year)>1933), 'Year before SEC.'
    assert(int(year)<=datetime.now().year), 'Year in future'
    assert(0<int(quarter)<5), 'Invalid quarter.'
    
    
    url = 'https://www.sec.gov/Archives/edgar/full-index/{:}/QTR{:}/master.idx'
    url = url.format(int(year),int(quarter))
    return pd.read_csv(url,sep='|',
                       skip_blank_lines=True,
                       skiprows=[0,1,2,3,4,5,6,7,8,10],
                       parse_dates=['Date Filed'])
def get_filings(CIK,page:int=0,n_records:int=100,type_=''):
    assert(type(page)==int),'Page must be an int.'
    assert(type(n_records==int)), 'n_records must be an int.'
    
    ed_url = 'https://www.sec.gov/cgi-bin/browse-edgar'
    r = requests.get(ed_url,params = dict(
        action='getcompany',
        CIK=CIK,
        count=n_records,
        start=page*n_records,
        type=type_
    ))
    assert(r),r
    
    try:
        soup = BeautifulSoup(r.text,'lxml')
        
        # Look if request is valid
        if "Invalid parameter" in [h.text for h in soup.findAll('h1')]:
            return 'Invalid parameter'
    
        # Get table of filings and parse
        filings = list(soup.find(class_="tableFile2").find_all('tr'))
        header = filings.pop(0)# Drop first row which only contains file headers.
        res = []
        
        if len(filings)>0:
            for f in filings:
                f = list(f.findAll('td'))
                t_dict = dict()
                t_dict['Filings'] = f[0].text
                t_dict['Format'] = f[1].find('a').get('href')
                t_dict['Filing Date'] = f[3].text
                t_dict['File/Film Number'] = f[4]
                res += [t_dict]
                
            res = pd.DataFrame(res)
            res['Filing Date'] = pd.to_datetime(res['Filing Date'])
            return res
        else:
            return ('No results.')
    except:
        print('ERROR PARSING!')
        return r
def get_all_pages(CIK,n_records:int=100,type_=''):
    

    valid = True
    page = 0
    
    res = pd.DataFrame()
    while valid:
        tdf = get_filings(CIK=CIK,
                          page=page,
                          n_records=n_records,
                          type_=type_)
        
        if(type(tdf)!=pd.DataFrame):
            valid = False
            break
        else:
            page+=1
            res = pd.concat([res,tdf])
        
    return res.reset_index(drop=True)
def get_most_recent(CIK:str,n_records:int=100,type_:str='NPORT-P')->download_NPORT:
    '''CIK: this should be a CIK or Series number for the fund. For companies with multiple funds (E.g. Schwab)
            using the CIK will result in the NPORT for the fund that was filed most recently. Multiple ticker symbols
            can trade under the same CIK, but will have unique Seires numbers. Different classes of the same fund
            will chase a Series number if they are based on the same investment series.
       n_records: specifies the number of records to pull for each EDGAR query.
            The options as of 6/8 are 40,60,80,and 100. This is included for flexibility 
            in case EDGAR changes allowed options.
        type_: is the string to pass to EDGAR regarding type. It should always be NPORT-P, but I am
            making this a variable in case the name changes in the future.
            
        Based on the series ID, this will return a download_NPORT object. Using this object, you can download
        a list of securities in the form of a pandas datafram using the command `parseSecurities()`. You can 
        get a dictionary of general fund infor using `parseFundInfo()`. You can get the XMLetree by accesing
        `xml`.
        '''
    #pull the given url of the most recent.
    recent = (
        get_all_pages(CIK,n_records=n_records,type_=type_)
            .sort_values('Filing Date',ascending=False)
            .head(1)['Format'].values[0]
    )
    
    # The URL by default is the HTML file. We need to modify it to get the XML data.
    ## Strip HTML part of link
    recent = recent[:recent.rfind('/')]
    
    ## Add information to access XML
    link = "https://www.sec.gov{:}/primary_doc.xml".format(recent)
    
    return download_NPORT(link)

In [3]:
secs = pd.read_excel('Persichitte 2020-06-08.xlsx',sheet_name='Summary')

secs.rename(columns={'Row Labels':'class_ticker','Sum of Mkt Value':'Amount'},inplace=True)
secs = secs.merge(
    pd.read_pickle('Investment Company Series and Class Information.p'),
    on='class_ticker',
    how='inner'
)
secs

,class_ticker,Amount,reporting_file_number,cik,entity_name,entity_org_type,series_id,series_name,class_id,class_name,address_1,city,zip_code,state,address_2
0,BLOK,406.00,811-23108,0001633061,Amplify ETF Trust,30,S000061158,Amplify Transformational Data Sharing ETF,C000198131,Amplify Transformational Data Sharing ETF,310 S. HALE ST.,WHEATON,60187,IL,[NULL]
1,HLEMX,2111.38,811-07739,0001018170,HARDING LOEVNER FUNDS INC,30,S000004201,Harding Loevner Emerging Markets Portfolio,C000011821,Advisor,400 CROSSING BLVD.,BRIDGEWATER,08807,NJ,FOURTH FLOOR
2,JERTX,2961.54,811-01879,0000277751,JANUS INVESTMENT FUND,30,S000025889,Janus Henderson Global Real Estate Fund,C000077601,Class T,151 DETROIT STREET,DENVER,80206,CO,[NULL]
3,MDYG,887.04,811-08839,0001064642,SPDR SERIES TRUST,30,S000006987,SPDR(R) S & P 400 Mid Cap Growth ETF,C000019040,SPDR(R) S & P 400 Mid Cap Growth ETF,ONE LINCOLN STREET,BOSTON,02111,MA,[NULL]
4,MDYV,518.80,811-08839,0001064642,SPDR SERIES TRUST,30,S000006988,SPDR(R) S & P 400 Mid Cap Value ETF,C000019041,SPDR(R) S & P 400 Mid Cap Value ETF,ONE LINCOLN STREET,BOSTON,02111,MA,[NULL]
5,MFAEX,1127.51,811-22449,0001496998,AMERICAN FUNDS MORTGAGE FUND,30,S000030190,AMERICAN FUNDS MORTGAGE FUND,C000092906,Class F-1,6455 IRVINE CENTER DRIVE,IRVINE,92618,CA,[NULL]
6,PCLAX,2535.53,811-05028,0000810893,PIMCO FUNDS,30,S000028928,PIMCO CommoditiesPLUS Strategy Fund,C000088650,Class A,650 NEWPORT CENTER DRIVE,NEWPORT BEACH,92660,CA,[NULL]
7,SCHC,138.41,811-22311,0001454889,SCHWAB STRATEGIC TRUST,30,S000026638,Schwab International Small-Cap Equity ETF,C000079985,Schwab International Small-Cap Equity ETF,211 MAIN STREET,SAN FRANCISCO,94105,CA,[NULL]
8,SCHF,6253.05,811-22311,0001454889,SCHWAB STRATEGIC TRUST,30,S000026637,Schwab International Equity ETF,C000079984,Schwab International Equity ETF,211 MAIN STREET,SAN FRANCISCO,94105,CA,[NULL]
9,SCHG,9959.96,811-22311,0001454889,SCHWAB STRATEGIC TRUST,30,S000026633,Schwab U.S. Large-Cap Growth ETF,C000079980,Schwab U.S. Large-Cap Growth ETF,211 MAIN STREET,SAN FRANCISCO,94105,CA,[NULL]


In [40]:
class aggregate_SID_holdings:
    def __init__(
        self,
        df:pd.DataFrame,
        sid_column:str,
        amount_column:str,
        n_records:int=100,
        type_:str='NPORT-P'
    ):
        print('Importing holdings by Series IDs')
        tdf = df[[sid_column,amount_column]]
        all_NPORT = pd.DataFrame()
        
        for i in tdf.index:
            sid = tdf.iloc[i][sid_column]
            print('------------------\n',sid)
            
            try:
                # Create a running list of all security info
                NPORT = get_most_recent(sid).parseSecurities()
                print(NPORT.shape[0],'securities downloaded.')
                
                NPORT['Source'] = sid
                
                ## Create columns for cusips with NA
                # Replace 'N/A' values
                NPORT['cusip_clean'] = NPORT['cusip']
                mask = (NPORT['cusip_clean']=='N/A') | (NPORT['cusip_clean']=='000000000') | NPORT['cusip_clean'].isna()
                NPORT.loc[mask,'cusip_clean'] = NPORT.loc[mask,'lei']
                
                # Replace where N/A is in 'lei'
                mask = (NPORT['cusip_clean']=='N/A') | (NPORT['cusip_clean']=='000000000') | NPORT['cusip_clean'].isna()
                NPORT.loc[mask,'cusip_clean'] = NPORT.loc[mask,'title']
                
                
                ## Add column for merge with absolute values in portfolio
                try:
                    NPORT['PortfolioValue'] = NPORT['pctVal'] * tdf.iloc[i][amount_column] / 100
                except:
                    print('Error determing value of individual holdings.')
                    NPORT['PortfolioValue'] = np.nan
                
                
                all_NPORT = pd.concat([all_NPORT,NPORT],sort=False)
            except:
                print('Unable to download securities.')
                
                
        # Store results in the class object
        self.all_NPORT = all_NPORT
        
        print('------------------\nComplete\n------------------')
        return

holds = aggregate_SID_holdings(secs,amount_column='Amount',sid_column='series_id')

Importing holdings by Series IDs
------------------
 S000061158
Unable to download securities.
------------------
 S000004201
76 securities downloaded.
------------------
 S000025889
61 securities downloaded.
------------------
 S000006987
235 securities downloaded.
------------------
 S000006988
297 securities downloaded.
------------------
 S000030190
517 securities downloaded.
------------------
 S000028928
885 securities downloaded.
------------------
 S000026638
2128 securities downloaded.
------------------
 S000026637
1487 securities downloaded.
------------------
 S000026633
352 securities downloaded.
------------------
 S000030518
96 securities downloaded.
------------------
 S000026635
506 securities downloaded.
------------------
 S000026634
411 securities downloaded.
------------------
 S000006990
336 securities downloaded.
------------------
 S000006974
455 securities downloaded.
------------------
 S000005911
508 securities downloaded.
------------------
 S000005912
1994 

### Confirmation

To confirm the holding value is calculated correctly, I am going to sum by SID and compare with the inputs.

<font color='red'>There is a problem with this. I'm leavining it in until I can research it further.</font>

In [44]:
holds.all_NPORT.groupby('Source')['PortfolioValue'].sum().sort_values()

Source
S000026638     140.520262
S000006988     526.450625
S000006974     641.294602
S000029442     775.284290
S000006990     786.459782
S000006987     896.002084
S000030190    1304.252567
S000026634    1934.233207
S000004201    2113.870781
S000030518    2223.420209
S000028928    2933.730861
S000025889    2949.527994
S000012902    3567.947118
S000005912    5715.789496
S000026637    6296.287152
S000005911    6569.702904
S000026635    8799.743098
S000026633    9951.310622
Name: PortfolioValue, dtype: float64

In [48]:
holds.all_NPORT.groupby('Source')['pctVal'].sum().sort_values()

Source
S000025889     99.594400
S000005911     99.797857
S000026634     99.819027
S000026633     99.913159
S000004201    100.117969
S000030518    100.257482
S000012902    100.573546
S000026635    100.584241
S000026637    100.691457
S000029442    100.884109
S000006987    101.010336
S000006988    101.474677
S000005912    101.500716
S000026638    101.524645
S000006990    103.299417
S000006974    104.583343
S000030190    115.675477
S000028928    115.704837
Name: pctVal, dtype: float64

In [46]:
secs[['series_id','Amount']].sort_values('Amount')

,series_id,Amount
7,S000026638,138.41
0,S000061158,406.00
4,S000006988,518.80
14,S000006974,613.19
13,S000006990,761.34
17,S000029442,768.49
3,S000006987,887.04
5,S000030190,1127.51
12,S000026634,1937.74
1,S000004201,2111.38


## Summary information and export to webpage

The following section uses methods from the class to export to a webpage.

In [56]:
cusips = holds.all_NPORT.fillna(0).groupby('cusip_clean',as_index=False)['PortfolioValue'].agg(['count','sum'])

# Get just the first record for CUSIPS that are duplicated.
# This prevents an accidental one to many join.
mask = ~holds.all_NPORT['cusip_clean'].duplicated() 

cusips = cusips.merge(holds.all_NPORT[mask],on='cusip_clean')
cusips.sort_values('sum',ascending=False)

,cusip_clean,count,sum,name,lei,title,cusip,identifiers,balance,units,...,isRestrictedSec,fairValLevel,securityLending,currencyConditional,Source,PortfolioValue,issuerConditional,debtSec,derivativeInfo,repurchaseAgrmt
4654,594918104,2,1193.610885,Microsoft Corp,INR2EJN1ERAN0W5ZP974,Microsoft Corp,594918104,\n,4867148.00,NS,...,N,1,\n,NaN,S000026633,875.146323,NaN,NaN,NaN,NaN
275,037833100,2,1125.462088,Apple Inc,HWUPKR0MPOU8FGXBT394,Apple Inc,037833100,\n,2664707.00,NS,...,N,1,\n,NaN,S000026633,808.441737,NaN,NaN,NaN,NaN
183,023135106,2,760.602442,Amazon.com Inc,ZXTILKJKG63JELOEG630,Amazon.com Inc,023135106,\n,265678.00,NS,...,N,1,\n,NaN,S000026633,555.447679,NaN,NaN,NaN,NaN
1785,30303M102,2,447.068307,Facebook Inc,BQ4BKCS1HXDV9HN80Z93,Facebook Inc,30303M102,\n,1535075.00,NS,...,N,1,\n,NaN,S000026633,327.911723,NaN,NaN,NaN,NaN
4091,549300F9QJIJF2GM8419,1,423.841132,PIMCO FUNDS,549300F9QJIJF2GM8419,PIMCO ST FLOATING NAV PORT IV MUTUAL FUND,000000000,\n,37674486.66,PA,...,N,1,\n,NaN,S000028928,423.841132,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7749,LOW SU GASOIL G DEC20 IFEU 20201210,1,-20.914144,N/A,N/A,LOW SU GASOIL G DEC20 IFEU 20201210,000000000,\n,1623.00,NC,...,N,1,\n,NaN,S000028928,-20.914144,NaN,NaN,\n,NaN
159,01F032641,1,-23.291188,UMBS PASS THRU POOLS,N/A,FNMA TBA 30 YR 3.5 SINGLE FAMILY MORTGAGE,01F032641,\n,-19000000.00,PA,...,N,2,\n,NaN,S000028928,-23.291188,NaN,\n,NaN,NaN
6763,BRENT CRUDE FUTR DEC21 IFEU 20211029,1,-30.426046,N/A,N/A,BRENT CRUDE FUTR DEC21 IFEU 20211029,000000000,\n,3574.00,NC,...,N,1,\n,NaN,S000028928,-30.426046,NaN,NaN,\n,NaN
6998,COMM SWAP TBL/CSIXTR GST,1,-34.773202,N/A,N/A,COMM SWAP TBL/CSIXTR GST,000000000,\n,1.00,NC,...,N,2,\n,NaN,S000028928,-34.773202,NaN,NaN,\n,NaN


In [50]:
cusips.sort_values('sum',ascending=False)

,count,sum
cusip_clean,,
594918104,2,1193.610885
037833100,2,1125.462088
023135106,2,760.602442
30303M102,2,447.068307
549300F9QJIJF2GM8419,1,423.841132
...,...,...
LOW SU GASOIL G DEC20 IFEU 20201210,1,-20.914144
01F032641,1,-23.291188
BRENT CRUDE FUTR DEC21 IFEU 20211029,1,-30.426046


0      False
1      False
2      False
3      False
4      False
       ...  
671    False
672    False
673    False
674    False
675    False
Name: cusip, Length: 11051, dtype: bool

In [ ]:
holds.all_NPORT['cusip'].duplicated